In [ ]:
from get_inference_service_url import get_inference_service_url
infer_url = get_inference_service_url()

In [ ]:
# how many requests to send
MAX_REQUESTS=50

In [ ]:
# Function to send a single REST request
def rest_request(data):
    json_data = {
        "inputs": [
            {
                "name": "pixel_values",
                "shape": data.shape,
                "datatype": "FP32",
                "data": data.tolist()
            }
        ]
    }

    start_time = time.time()
    response = requests.post(infer_url, json=json_data, verify=True)
    end_time = time.time()

    response_time = end_time - start_time
    print(f"Response time: {response_time:.6f} seconds")

    return response

In [ ]:
# Function to handle a single prediction request
def process_request():
    response = rest_request(pixel_values)
    if response.status_code == 200:
        prediction_data = response.json()
    else:
        print(f"Request failed with status code: {response.status_code}")
        return None
    

In [ ]:
import os
import glob
import pickle
import time
import requests
from concurrent.futures import ThreadPoolExecutor

def find_pkl_files(directory):
    """Find all .pkl files in the specified directory."""
    pkl_files = []
    for root, dirs, files in os.walk(directory):
        pkl_files.extend(glob.glob(os.path.join(root, '*.pkl')))
    return pkl_files


def process_file(file):
    """Process a single .pkl file."""
    try:
        with open(file, 'rb') as f:
            image_preprocessed = pickle.load(f)
            pixel_values = image_preprocessed["pixel_values"]
            rest_request(pixel_values)
            print(f"Processed: {os.path.basename(file)}")
    except Exception as e:
        print(f"Error processing {file}: {e}")

# Main script
directory = "/opt/app-root/src/ai-mazing-race/lab-materials/06/06-wildfire-sample"
pkl_files = find_pkl_files(directory)

# Limit the processing to the first 50 .pkl files
limited_pkl_files = pkl_files[:MAX_REQUESTS]

start_time = time.time()

# Process files in parallel
with ThreadPoolExecutor(max_workers=8) as executor:
    executor.map(process_file, limited_pkl_files)

end_time = time.time()

# Print summary
elapsed_time = end_time - start_time
print(f"Total files processed: {len(limited_pkl_files)}")
print(f"Time taken: {elapsed_time:.2f} seconds")
